In [1]:
from pprint import pprint
import numpy as np
import param
import paramnb
import holoviews as hv
import holoviews.plotting.mpl

Have added `depends` decorator. Supports: 
  * declaring dependencies on parameters (value, metadata)
  * declaring dependencies on methods
  * memoizing on arguments+parameters declared as dependencies
  * declaring that a method is "viewable"
  * specifying that dependency should be eagerly evaluated (default is just declaring dependency, with no automatic action)

In [2]:
class ImageExample(param.Parameterized):

    color = param.Color(default='#000000', precedence=0)

    element = param.ObjectSelector(default=hv.Curve,
                                   objects=[hv.Curve, hv.Scatter, hv.Area],
                                   precedence=0)

    amplitude = param.Number(default=2, bounds=(2, 6))
    
    frequency = param.Number(default=2, bounds=(1, 10))

    @param.depends("frequency", cache=True)
    def big_computation(self, something):
        print("big computation")
        return something*np.sin(np.linspace(0, np.pi*self.frequency))
 
    @param.depends("amplitude","big_computation","color","element",viewable=True)
    def update(self, something=1.0):
        return self.element(self.amplitude*self.big_computation(something=something),
                            vdims=[hv.Dimension('y', range=(-5, 5))])(style=dict(color=self.color))


## "subscribing to a method" and "watching a parameter"

Can subscribe to parameters on which a method depends:

In [3]:
another_example = ImageExample()

In [4]:
def f1(change):
    print("f1")
    pprint(change)

In [5]:
# new method (TODO: should be onparam namespace obj)
another_example.param.subscribe("big_computation",f1)

In [6]:
another_example.frequency=2.4

f1
Change(what='value', attribute='frequency', obj=ImageExample(amplitude=2, color='#000000', element=<class 'holoviews.element.chart.Curve'>, frequency=2.4, name='ImageExample01096'), cls=<class '__main__.ImageExample'>, old=2, new=2.4)


GUIs passed `another_example` can ask for its viewables, and will subscribe to them so it knows whenever relevant parameters change.

In [7]:
another_example.param.viewables()

['update']

Can also watch one particular parameter:

In [8]:
an_example = ImageExample()

In [9]:
def f2(change):
    print("f2")
    pprint(change)

In [10]:
# new method (TODO: should be on param namespace obj)
an_example.param.watch("color",fn=f2)

In [11]:
an_example.color = "#FFFFFF"

f2
Change(what='value', attribute='color', obj=ImageExample(amplitude=2, color='#FFFFFF', element=<class 'holoviews.element.chart.Curve'>, frequency=2, name='ImageExample01097'), cls=<class '__main__.ImageExample'>, old='#000000', new='#FFFFFF')


## gui tks can look for viewables and can subscribe to dependencies

Ask Parameterized object for viewables. For each viewable, ask for list of parameters the viewable depends on, then adds subscriber/callbacl to those parameters. Callback calls the viewable method and updates the corresponding (trait/property).

In [12]:
example = ImageExample(name="HoloViews Example")

In [13]:
def my_renderer(x):
    return hv.Store.renderers['matplotlib'](x)[0],(300,300)

In [14]:
paramnb.Widgets(example, renderers={"update":my_renderer})

big computation


## eager dependencies

In [15]:
class ImageExample2(ImageExample):
    
    @param.depends("amplitude", eager=True)
    def red_or_black(self):
        if self.amplitude>=4:
            self.color = "#FF0000"
        else:
            self.color = "#000000"    

    # eager=False is the default; this method won't be called just because amplitude changes
    @param.depends("amplitude", eager=False)
    def set_color2(self):
        raise               

In [16]:
example2 = ImageExample2()

In [17]:
paramnb.Widgets(example2, renderers={"update":my_renderer})

big computation


In [18]:
# color changes to red
example2.amplitude=4.1

In [19]:
# color back to black
example2.amplitude=3.0

Note the reason the GUI happens to update when amplitude is modified is because the GUI is already watching amplitude, as amplitude is a dependency of "update", which is viewable so the GUI has subscribed to it.

GUI needs a refresh method that can be added as a subscriber to all parameters that are dependencies (of methods). I think it's not as simple as just watching all parameters - there are parameters of subobjects too (see below), i.e. otherwise gui needs to watch all parameters and all parameters of all subobjects. Same if gui's to reflect changes from outside the gui.

If we could declare that a method modifies a parameter, then the GUI could could e.g. look at all methods of an object and get all parameters on which all methods depend, and watch them for changes.

## dependencies on parameters of subobjects

Can depend on parameters of subobjects using `.`, e.g. `subobject.x`.

In [20]:
class X(param.Parameterized):
    z = param.Number(default=1)

objects = [X(),X()]

In [21]:
class AThing(param.Parameterized):
    x = param.Number(default=1, bounds=(0,10))
    y = param.ObjectSelector(default=objects[0],objects=objects)
    
class AThing2(AThing):
    pass

In [22]:
class ImageExample3(ImageExample2):

    something = param.ClassSelector(AThing,default=AThing2())    
    
    @param.depends("something.x",eager=True)
    def yellow_or_black(self): 
        if self.something.x>3:
            self.color = "#FFFF00"
        else:
            self.color = "#000000"
 

In [23]:
example3 = ImageExample3()

In [24]:
paramnb.Widgets(example3, renderers={"update":my_renderer})

big computation


## dependencies on metadata

Can depend on things about parameter other than its value, e.g. bounds, constant, etc. Using `:`, e.g. `parameter:constant`.

In [25]:
class ImageExample4(ImageExample):
    
    @param.depends("amplitude:constant", eager=True)
    def report_amplitude_constant(self):
        print("set amplitude.constant=%s"%self.params("amplitude").constant)


In [26]:
example4 = ImageExample4()

In [27]:
example4.amplitude=2.1

In [28]:
example4.params('amplitude').constant=True

set amplitude.constant=True


In [29]:
example4.params('amplitude').constant=False

set amplitude.constant=False


Can also watch parameter metadata:

In [30]:
def f3(change):
    print("f3")
    pprint(change)

In [31]:
example5 = ImageExample4()

In [32]:
# TODO: should support same string spec
example5.param.watch("color","constant",fn=f3)

In [33]:
example5.color="#FFFFFF"

In [34]:
example5.params('color').constant=True

f3
Change(what='constant', attribute='color', obj=None, cls=<class '__main__.ImageExample'>, old=False, new=True)


In [35]:
example5.params('color').constant=False

f3
Change(what='constant', attribute='color', obj=None, cls=<class '__main__.ImageExample'>, old=True, new=False)


## caching

Caching of method result based on arguments and declared parameter dependencies. I just did a quick hack so I have a 'whole demo', but hv has memoize already (TODO: link to it).

In [36]:
some_example = ImageExample(name='HoloViews Example')

In [37]:
# computes
some_example.update()

big computation


:Curve   [x]   (y)

In [38]:
# doesn't re-compute
some_example.update()

:Curve   [x]   (y)

In [39]:
# computes
some_example.update(something=1.1)

big computation


:Curve   [x]   (y)

In [40]:
# doesn't re-compute
some_example.update(something=1.1)

:Curve   [x]   (y)

In [41]:
# computes
some_example.frequency+=0.1
some_example.update()

big computation


:Curve   [x]   (y)

------

## Technical overview of dependencies

* dependencies are stored on the method (nothing but the method exists when decorator is defined)
* `Parameterized.params_depended_on(slf_or_cls, mthd)`: can ask Parameterized for "parameters on which a method depends": get back leaves that are Parameters (i.e. flat list of parameters)
* `Parameterized.viewables(slf_or_cls)`: get list of 'viewable' methods


Notes:
* I think `viewable` should be a separate decorator, and it should be for declaring things about the return type.
* I think it should be possible to declare also that a method modifies parameters. Would allow to detect cycles, and would allow e.g. a gui to know which parameters to watch if it always wants to stay up to date.
* (Yes, things should be in param namespace i.e. on `Parameters`)

## Technical overview of subscribing/watching

  * `subscribers` added as `Parameter` attribute/metadata. Though I think it's different from other parameter metadata...so this should change or at least be described differently. Unlike `Parameter` metadata, but like "the parameter value", `subscribers` can be set per instance as well as on the Parameter itself (i.e. per class).

  * `subscribers` is a dictionary `{data:[*callables]}`, where "data" can be `"value"` or any other Parameter metadata (e.g. `"constant"`, `"bounds"`, etc).

  * callable gets the "change": 
  ```
  {'cls': (class owning the parameter),
   'obj': (object parameter was set on; could be None),
   'data': (what data changed, e.g. value, bounds, constant, etc), 
   'attribute': (owning class's name for parameter),
   'old': (previous value),
   'new': (new value)}
   ```

  * watching changes to parameter value setting: monitor `Parameter.__set__`, which gets the relevant Parameterized instance (if any) and Parameterized class.

  * watching changes to parameter metadata not so obvious. Watching parameter metadata setting is currently done by adding a `__setattr__` method to Parameter. Alternatively we could have ParameterParameter (i.e. make parameter attributes be descriptors). Then we could validate parameter metadata too ;) (Also note that by the time an attribute is set on a Parameter, there's no associated Parameterized instance (of course - Parameter object is per class), so making parameter metadata vary per instance would require larger changes in param.)

  * On parameterized instantiation (`Parameterized.__init__`), for each method found to have eager dependencies, those dependencies are watched (annoying to have more done on every parameterized initialization).
  
  * Other supporting things Parameterized has grown: 
    * `params_depended_on(mthd)`: get flat list of parameters a method depends on
    * `viewables()`: get list of 'viewable' methods
    * turn a "depends string spec" into a real thing e.g.
      * `"x"`: depend on value of parameter `x` if `x` is a parameter, or method `x` if `x` is a method
      * `"x:value"`: depend on value of parameter `x` (value written explicitly - equivalent to `x` if `x` is a parameter)
      * `"x:constant"`: depend on `constant` attribute of parameter `x` (instance or class: `cls.params('x').constant`)
      * `"some_method"`: depend on method `some_method` (instance method: `self.some_method`; class method not yet implemented)
      * `"x.y"`: depend on attribute `y` of attribute `x` (`y` could be parameter or method).

Notes:

  * Changes to mutable objects not detected. E.g. changing a list in a parameter. Only *setting* is detected right now.
  * Imagine you're watching `x.y`. E.g. you have done `@depends("x.y", eager=True)`. If `y` is set you hear about it...but what if `x` is changed to a different object? You're don't hear about `y` changing if the new object has a different value for `y`, and you never hear about any future changes because the hooks were installed in the original `x` instance...    
  * I think it shouldn't be `@depends(..., viewable=True)`; there should be a general `@output(...)` kind of decorator where (things about) the return type can be declared.

## Technical overview of caching

TODO

## Confusing things a user could do

### set up a cycle

In [42]:
import time
import math
import colorsys

In [43]:
class ImageExampleCycle(ImageExample):
    
    # set up a cycle
    @param.depends("color",eager=True)
    def cycle(self):
        global some_count
        if some_count <= 75:
            some_count += 1            
            self.color = '#%02x%02x%02x' % tuple([int(math.floor(255*i)) for i in colorsys.hsv_to_rgb((some_count%10)/10.0,1.0,1.0)])    

In [44]:
some_count = 0
paramnb.Widgets(ImageExampleCycle(), renderers={"update":my_renderer}, on_init=True)

big computation


### declare dependencies on methods, but forget to call them

In [45]:
class ImageExampleMistake(ImageExample):
    
    @param.depends("amplitude", eager=True)
    def red_or_black(self):
        if self.amplitude>=4:
            self.color = "#FF0000"
        else:
            self.color = "#000000"
        
    @param.depends("something.x",eager=True)
    def yellow_or_black(self): 
        if self.something.x>3:
            self.color = "#FFFF00"
        else:
            self.color = "#000000"
            
    # declares dependencies on methods, but doesn't actually call them; what does this mean?
    @param.depends("red_or_black","yellow_or_black",eager=True)
    def blue(self):
        if self.amplitude>5:
            self.color = "#0000FF"

In [46]:
example = ImageExample()

In [47]:
example.amplitude = 5.1
#assert example.color == '#0000FF',example.color

### declare dependency on method that ultimately depends on no parameter

In [48]:
# TODO

## other issues

* `@depends(fn1,fn2 eager=True)`: this just means that your method will be called if parameters on which fn1 and fn2 depend are set - your method is still responsible for calling fn1 and fn2 (and doing so in the right order, if it matters).
* Hopefully when parameter metadata changes, e.g. parameter set to constant, our gui tks can handle widget change (i.e. they are ok with not just value changing once the widget has been created...e.g. hopefully they don't need to replace the widget with a different one, or anything like that)
* Maybe just make `@depends(parameter)` mean anything about parameter (value or metadata). And maybe don't support specifics (i.e. no `:`). Or maybe have something like `:*`.

-----

## Tests/checks

Ignore...

In [49]:
class AThing(param.Parameterized):
    x = param.Number(default=1, bounds=(0,10))
    y = param.ObjectSelector(default=objects[0],objects=objects)
    
    @param.depends("y:constant", eager=True)
    def some_method(self):
        self.some_method_was_called = True
    
class AThing2(AThing):
    pass

class AThing3(AThing):
    pass

In [50]:
class TestExample(param.Parameterized):

    color = param.Color(default='#000000', precedence=0)

    element = param.ObjectSelector(default=hv.Curve,
                                   objects=[hv.Curve, hv.Scatter, hv.Area],
                                   precedence=0)

    amplitude = param.Number(default=2, bounds=(2, 6))
    
    frequency = param.Number(default=2, bounds=(1, 10))
    
    something = param.ClassSelector(AThing,default=AThing2())

    @param.depends("frequency", cache=True)
    def big_computation(self, something):
        print("big computation")
        return something*np.sin(np.linspace(0, np.pi*self.frequency))
 
    @param.depends("amplitude","big_computation","color","element",viewable=True)
    def update(self, something=1.0):
        return self.element(self.amplitude*self.big_computation(something=something),
                                  vdims=[hv.Dimension('y', range=(-5, 5))])(style=dict(color=self.color))
    
    @param.depends("amplitude", eager=True)
    def red_or_black(self):
        if self.amplitude>=4:
            self.color = "#FF0000"
        else:
            self.color = "#000000"
        
    @param.depends("something.x",eager=True)
    def yellow_or_black(self): 
        if self.something.x>3:
            self.color = "#FFFF00"
        else:
            self.color = "#000000"
            
            
#    @depends("something.some_method",eager=True)
#    def test123(self): 
#        print("test123")            
    #@modifies=(element")
    @param.depends("something.x:constant",eager=True)
    def test789(self):
        print("test789")
        self.params("element").constant=self.something.params("x").constant
        #self.param_set("element","constant", self.something.param_get("x","constant"))
            

    # eager=False is the default; this method won't be called just because amplitude changes
    @param.depends("amplitude", eager=False)
    def set_color2(self):
        raise

In [51]:
test = TestExample()

In [52]:
assert test.color == '#000000'

In [53]:
test.param.params_depended_on("red_or_black")

[PInfo(inst=TestExample(amplitude=2, color='#000000', element=<class 'holoviews.element.chart.Curve'>, frequency=2, name='TestExample01857', something=AThing2(name='AThing201858', x=1, y=X(name='X01648', z=1))), cls=<class '__main__.TestExample'>, name='amplitude', pobj=<param.Number object at 0x120105198>, what='value')]

In [54]:
test.amplitude = 4.1
assert test.color == '#FF0000'

In [55]:
test.something.x = 2
assert test.color == '#000000'

In [56]:
test.something.x = 4
assert test.color == '#FFFF00'

In [57]:
assert test.params('element').constant is False

In [58]:
test.something.params("x").constant=True
assert test.params('element').constant is True

test789


In [59]:
test.amplitude = 2.0
assert test.color == '#000000'

In [60]:
assert len(test.param.params_depended_on("update")) == 4

In [61]:
_ = example.update()

big computation


In [62]:
assert len(test.param.params_depended_on("update")) == 4

In [63]:
at = AThing()
assert at.params("y").constant == False
assert not hasattr(at,"some_method_was_called")
at.params("y").constant=True
assert at.some_method_was_called == True

-----

In [64]:
# TODO: decide about class methods

In [65]:
class TestExample(param.Parameterized):

    color = param.Color(default='#000000', precedence=0)

    amplitude = param.Number(default=2, bounds=(2, 6))
    
    @param.depends("amplitude", eager=True) # eager applies only to instances at the moment
    @classmethod    
    def red_or_black(cls):
        print("red_or_black")
        if cls.amplitude>=4:
            cls.color = "#FF0000"
        else:
            cls.color = "#000000"            

In [66]:
print(TestExample.amplitude, TestExample.color)

2 #000000


In [67]:
TestExample.amplitude=4.1

In [68]:
print(TestExample.amplitude, TestExample.color)

4.1 #000000
